# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
!pip install wandb -qU

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [5]:
diamonds.shape

(53940, 10)

In [6]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [8]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:34:25] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [9]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


### Incorporate validation

In [10]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [11]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058
[30]	train-rmse:464.38845	validation-rmse:537.01895


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:34:25] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [12]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:34:26] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[102]	train-rmse:376.20429	validation-rmse:532.59813


In [13]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:34:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [14]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [15]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



## Experiments

In [16]:
# Log in to your W&B account
import wandb

wandb.login()

wandb: Currently logged in as: shreya18 (practicum-class). Use `wandb login --relogin` to force relogin


True

In [31]:
from sklearn.model_selection import GridSearchCV
import math
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [21]:
y_train.max()

price    18818
dtype: int64

In [19]:
# Initialize W&B for experiment tracking
wandb.init(project="xgboost_5_experiments")


# Define five distinct hyperparameter sets
hyperparameter_sets = [
    {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'enable_categorical': True},
    {'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.05, 'subsample': 0.8, 'enable_categorical': True},
    {'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.3, 'subsample': 0.9,  'enable_categorical': True},
    {'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.15, 'subsample': 1.0, 'enable_categorical': True},
    {'n_estimators': 120, 'max_depth': 7, 'learning_rate': 0.07, 'subsample': 0.85, 'enable_categorical': True},
]

# Run five experiments
results = []

for idx, params in enumerate(hyperparameter_sets):
    # Log each experiment with W&B
    with wandb.init(reinit=True, name=f"experiment_{idx + 1}"):
        # Create a model with the current hyperparameters
        model = XGBRegressor(**params)

        # Train the model
        model.fit(X_train, y_train)

        # Predict on the test set
        y_pred = model.predict(X_test)

        # Calculate mse
        mse = mean_squared_error(y_test, y_pred)

        # Log the hyperparameters and accuracy
        wandb.log({
            "hyperparameters": params,
            "mean_squared_error": mse,
        })

        # Store the results
        results.append({
            "experiment": idx + 1,
            "mean_squared_error": mse,
            "hyperparameters": params
        })

# Output the results of the experiments
print("Results of 5 Experiments:")
for result in results:
    print(result)

wandb.finish()  # End the W&B run

mean_squared_error,▁
mean_squared_error,275858.35683


mean_squared_error,▁
mean_squared_error,279000.34843


mean_squared_error,▁
mean_squared_error,315173.28527


mean_squared_error,▁
mean_squared_error,302773.89442


mean_squared_error,▁
mean_squared_error,281797.25337


Results of 5 Experiments:
{'experiment': 1, 'mean_squared_error': 275858.3568298423, 'hyperparameters': {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 1.0, 'enable_categorical': True}}
{'experiment': 2, 'mean_squared_error': 279000.34843107575, 'hyperparameters': {'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.05, 'subsample': 0.8, 'enable_categorical': True}}
{'experiment': 3, 'mean_squared_error': 315173.2852692758, 'hyperparameters': {'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.3, 'subsample': 0.9, 'enable_categorical': True}}
{'experiment': 4, 'mean_squared_error': 302773.8944247214, 'hyperparameters': {'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.15, 'subsample': 1.0, 'enable_categorical': True}}
{'experiment': 5, 'mean_squared_error': 281797.25337050535, 'hyperparameters': {'n_estimators': 120, 'max_depth': 7, 'learning_rate': 0.07, 'subsample': 0.85, 'enable_categorical': True}}


## Randomized Search

In [20]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as st

# Initialize W&B for experiment tracking
wandb.init(project="xgboost_random_search")

# Define a hyperparameter space for Random Search
param_distributions = {
    'n_estimators': st.randint(50, 200),
    'max_depth': st.randint(3, 10),
    'learning_rate': st.uniform(0.01, 0.3),
    'subsample': st.uniform(0.7, 0.3),
    'colsample_bytree': st.uniform(0.7, 0.3),
    'enable_categorical': [True],
}

# Configure RandomizedSearchCV with XGBRegressor
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(),
    param_distributions=param_distributions,
    n_iter=5,  # Number of random experiments
    scoring='neg_mean_squared_error',  # Evaluation metric for regression
    cv=3,  # Cross-validation folds
    n_jobs=-1,  # Parallel processing
    random_state=42,  # For reproducibility
)

# Run Random Search
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

# Log the best hyperparameters and MSE to W&B
wandb.log({
    "best_hyperparameters": random_search.best_params_,
    "mean_squared_error": mse,
})

print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Mean Squared Error: {mse}")

wandb.finish()  # End the W&B run

Best Hyperparameters: {'colsample_bytree': 0.7468055921327309, 'enable_categorical': True, 'learning_rate': 0.05679835610086079, 'max_depth': 5, 'n_estimators': 137, 'subsample': 0.8001125833417065}
Mean Squared Error: 292918.1948454305


mean_squared_error,▁
mean_squared_error,292918.19485


## Findings

These results from five experiments reveal differences in performance across varying hyperparameter combinations, highlighting the impact of hyperparameters on model accuracy. Here's a breakdown of the findings and some insights into the patterns observed:
General Observations:

    Mean Squared Error (MSE): MSE is a key metric for regression, with lower values indicating better model performance. In these results, the MSE ranges from 275,858 to 315,173, indicating varying degrees of model performance.

Specific Observations:

    Experiment 1:
        MSE: 275,858.36
        Hyperparameters: n_estimators=100, max_depth=6, learning_rate=0.1, subsample=1.0
        Observation: This experiment has the lowest MSE, suggesting that these hyperparameters produce the most accurate results among the experiments. A moderate learning rate and estimator count, combined with full subsample, seem to work well.

    Experiment 2:
        MSE: 279,000.35
        Hyperparameters: n_estimators=150, max_depth=8, learning_rate=0.05, subsample=0.8
        Observation: The MSE is slightly higher than Experiment 1, indicating a marginal decrease in accuracy. This suggests that increasing n_estimators and max_depth without a proportionate learning rate might reduce performance.

    Experiment 3:
        MSE: 315,173.29
        Hyperparameters: n_estimators=50, max_depth=4, learning_rate=0.3, subsample=0.9
        Observation: This experiment has the highest MSE, indicating lower accuracy. A high learning rate might lead to overfitting or unstable training. Additionally, a low estimator count and shallow depth could lead to underfitting.

    Experiment 4:
        MSE: 302,773.89
        Hyperparameters: n_estimators=200, max_depth=10, learning_rate=0.15, subsample=1.0
        Observation: This setup has a high MSE, suggesting that too many estimators and a high max_depth might lead to overfitting. A moderate learning rate helps, but the overall depth might contribute to instability.

    Experiment 5:
        MSE: 281,797.25
        Hyperparameters: n_estimators=120, max_depth=7, learning_rate=0.07, subsample=0.85
        Observation: This result is close to Experiment 2, with a slightly lower MSE, suggesting a more balanced hyperparameter setup. The moderate learning rate, subsample, and max_depth seem to work well.

Conclusions and Recommendations:

    Optimal Hyperparameters: Experiment 1 produced the best results, indicating that a moderate learning rate, estimator count, and depth, combined with full subsample, are likely optimal.
    Learning Rate Impact: High learning rates (as seen in Experiment 3) can lead to poor performance due to overfitting or unstable learning.
    Estimator Count and Depth: Both should be carefully balanced to avoid overfitting or underfitting. A high count and depth can lead to overfitting, while too low can result in underfitting.
    Subsample: Full subsample (1.0) appears to provide better results, likely due to more stable training.

In [32]:
##end of notebook

In [ ]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see